In [1]:
from collections import OrderedDict 
import yaml
import os

# Get existing specifications

## Extract paths

In [2]:
tags = [
#     'execution', 
#     'job', 
    'project', 
#     'storage', 
#     'system', 
#     'user',
#     'metric'
]

paths = {}
for tag in tags:
    api_file = os.path.join('..', 'rundeck', f'{tag}.yaml')
    with open(api_file, 'r') as af:
        api_dict = yaml.load(af, Loader = yaml.FullLoader)
    
    for path, api in api_dict['paths'].items():
        for method, content in api.items():
            new_path = {}
            
            # summary
            new_path['summary'] = content['summary']
            new_path['operationId'] = content['operationId']
            
            # tags
            if not 'tags' in content.keys():
                new_path['tags'] = [tag]
            else:
                new_path['tags'] = content['tags']
                
            # parameters
            if 'parameters' in content.keys():
                parameters = []
                request_body = {}
                for old_parameter in content['parameters']:
                    if old_parameter['in'] == 'body':
#                         request_body['name'] = old_parameter['name']
                        request_body['required'] = old_parameter['required']
                        
                        request_body['content'] = {
                            "application/json": {
                                "schema": {
                                    "type": "object"
                                }
                                
                        }}
                        if 'properties' in old_parameter['schema'].keys():
                            request_body['content']['application/json']['schema']['properties'] = old_parameter['schema']['properties']
                        new_path['requestBody'] = request_body
                    else:                 
                        new_parameter = {}
                        for k,v in old_parameter.items():
                            newtype = {}
                            if k == 'type':
                                newtype['type'] = v
                            elif k == "default":
                                newtype['default'] = v  
                            elif k == "enum":
                                newtype['enum'] = v
                            else:
                                new_parameter[k] = v
                            new_parameter['schema'] = newtype
                        parameters.append(new_parameter)
                    if len(parameters) > 0:
                        new_path['parameters'] = parameters
                
            # responses
            if 'responses' in content.keys():
                if '200' in content['responses'].keys():
                    new_path['responses'] = {
                        '200': {
                            'description': content['responses']['200']['description']
                        }
                    }
                    if 'schema' in content['responses'].keys():
                        new_path['responses']["schema"]: {
                            "application/json": {
                                "schema": content['responses']['200']['schema']}}
            if not 'responses' in new_path.keys():              
                new_path['responses'] = {
                    '200': {
                        'description': "Success !"
                    }
                }


            paths[path] = {}
            paths[path][method] = new_path

schemas_file = os.path.join('..', 'rundeck', 'definitions.yaml')
with open(schemas_file, 'r') as sf:
    schemas_dict = yaml.load(sf, Loader = yaml.FullLoader)


## write security schemes (not schemas !)

In [3]:
security_schemes = {
    'rundeck_auth': {
        "type": 'apiKey',
        "in": "header",
        "name": "X-Rundeck-Auth-Token"
    }  
}

## Create tags

In [4]:
alltags = set()
for path, api in paths.items():
    for method, action in api.items():
        for tag in action['tags']:
            alltags.add(tag)

tags = []
for tag in list(alltags):
    tags.append({
        "name": tag,
        "description": f"API endpoints to {tag}s" 
    })

## Generate external Docs

In [5]:
external_docs = {
    "description": "Combined API specifications for Rundeck",
    "url": "https://github.com/Lattice-Works/openlattice-rundeck"
}

# Combine in specifications

In [6]:
full_spec = {}
full_spec['openapi'] = "3.0.0"
full_spec['info'] = api_dict['info']
full_spec['paths'] = paths
full_spec['components'] = {}
full_spec['components']['securitySchemes'] = security_schemes
full_spec['components']['schemas'] = schemas_dict
full_spec['tags'] = tags
full_spec['externalDocs'] = external_docs

# Write to combined yaml

In [7]:
new_file = "./rundeck.yaml"
with open(new_file, 'w') as fl:
    yaml.dump(full_spec, fl, default_flow_style=False, sort_keys=False)